In [27]:
import pandas as pd
import numpy as np
import dateutil

In [64]:
# Load data from csv file
data = pd.read_csv('phone_data.csv')

The main columns in the file are:

*date:* The date and time of the entry

*duration:* The duration (in seconds) for each call, the amount of data (in MB) for each data entry, and the number of texts sent (usually 1) for each sms entry.

*item:* A description of the event occurring – can be one of call, sms, or data.

*month:* The billing month that each entry belongs to – of form ‘YYYY-MM’.

*network:* The mobile network that was called/texted for each entry.

*network_type:* Whether the number being called was a mobile, international (‘world’), voicemail, landline, or other (‘special’) number.

In [4]:
data.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [9]:
data.date.head()

0    15/10/14 06:58
1    15/10/14 06:58
2    15/10/14 14:46
3    15/10/14 14:48
4    15/10/14 17:27
Name: date, dtype: object

In [19]:
%time data['date'].apply(dateutil.parser.parse, dayfirst=True).head()

Wall time: 74.1 ms


0   2014-10-15 06:58:00
1   2014-10-15 06:58:00
2   2014-10-15 14:46:00
3   2014-10-15 14:48:00
4   2014-10-15 17:27:00
Name: date, dtype: datetime64[ns]

In [59]:
# Same thing can be achieved with lesser code but slower
%time pd.to_datetime(data.date).head()

Wall time: 0 ns


0   2014-10-15 06:58:00
1   2014-10-15 06:58:00
2   2014-10-15 14:46:00
3   2014-10-15 14:48:00
4   2014-10-15 17:27:00
Name: date, dtype: datetime64[ns]

In [65]:
data.date = data['date'].apply(dateutil.parser.parse, dayfirst=True)

In [55]:
# How many rows the dataset
data['item'].count()

830

In [56]:
# What was the longest phone call / data entry?
data['duration'].max()

10528.0

In [24]:
# How many seconds of phone calls are recorded in total?
data['duration'][data['item'] == 'call'].sum()

92321.0

In [25]:
# How many entries are there for each month?
data['month'].value_counts()

2014-11    230
2015-01    205
2014-12    157
2015-02    137
2015-03    101
Name: month, dtype: int64

# Aside

In [28]:
df=pd.DataFrame({
    'key1':['a','a','b','b','a'],
    'key2':['one','two','one','two','one'],
    'data1':np.random.randn(5),
    'data2':np.random.randn(5)
})

In [29]:
df

,key1,key2,data1,data2
0,a,one,-0.734694,1.610741
1,a,two,1.039071,-0.171322
2,b,one,-0.346890,-0.616630
3,b,two,1.055713,-0.284040
4,a,one,0.018204,0.229135


In [30]:
grouped=df['data1'].groupby(df['key1'])
print(grouped.size(),grouped.nunique(),grouped.count())

key1
a    3
b    2
Name: data1, dtype: int64 key1
a    3
b    2
Name: data1, dtype: int64 key1
a    3
b    2
Name: data1, dtype: int64


# Continuation

In [31]:
# Number of non-null unique network entries
data['network'].nunique()

9

The groupby() function returns a GroupBy object, but essentially describes how the rows of the original data set has been split. the GroupBy object .groups variable is a dictionary whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group. For example:

In [32]:
data.groupby(['month']).groups.keys()

dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

In [34]:
# Get the first entry for each month
data.groupby('month').first()

,index,date,duration,item,network,network_type
month,,,,,,
2014-11,0,15/10/14 06:58,34.429,data,data,data
2014-12,228,13/11/14 06:58,34.429,data,data,data
2015-01,381,13/12/14 06:58,34.429,data,data,data
2015-02,577,13/01/15 06:58,34.429,data,data,data
2015-03,729,12/02/15 20:15,69.000,call,landline,landline


In [46]:
# Get the sum of the durations per month
data.groupby('month')['duration'].sum().round(2)

month
2014-11    26639.44
2014-12    14641.87
2015-01    18223.30
2015-02    15522.30
2015-03    22750.44
Name: duration, dtype: float64

In [40]:
# Get the number of dates / entries in each month
data.groupby('month')['item'].unique()

month
2014-11    [data, call, sms]
2014-12    [data, call, sms]
2015-01    [data, sms, call]
2015-02    [data, call, sms]
2015-03    [call, data, sms]
Name: item, dtype: object

In [41]:
# What is the sum of durations, for calls only, to each network
data[data['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [42]:
# How many calls, sms, and data entries are in each month?
data.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [43]:
# How many calls, texts, and data are sent per month, split by network_type?
data.groupby(['month', 'network_type'])['date'].count()

month    network_type
2014-11  data             29
         landline          5
         mobile          189
         special           1
         voicemail         6
2014-12  data             30
         landline          7
         mobile          108
         voicemail         8
         world             4
2015-01  data             31
         landline         11
         mobile          160
         voicemail         3
2015-02  data             31
         landline          8
         mobile           90
         special           2
         voicemail         6
2015-03  data             29
         landline         11
         mobile           54
         voicemail         4
         world             3
Name: date, dtype: int64

In [44]:
data.groupby('month')['duration'].sum() # produces Pandas Series
data.groupby('month')[['duration']].sum() # Produces Pandas DataFrame

,duration
month,
2014-11,26639.441
2014-12,14641.870
2015-01,18223.299
2015-02,15522.299
2015-03,22750.441


The groupby output will have an index or multi-index on rows corresponding to your chosen grouping variables. To avoid setting this index, pass `as_index=False` to the groupby operation.

In [47]:
data.groupby('month', as_index=False).agg({"duration": "sum"})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


In [48]:
data.groupby('month', as_index=False)[['duration']].sum()

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


The aggregation functionality provided by the `agg()` function allows multiple statistics to be calculated per group in one calculation. Instructions for aggregation are provided in the form of a python dictionary or list. The dictionary keys are used to specify the columns upon which you’d like to perform operations, and the dictionary values to specify the function to run.

In [66]:
# Group the data frame by month and item and extract a number of stats from each group
data.groupby(['month', 'item']).agg({'duration': 'sum',      # find the sum of the durations for each group
                                     'network_type': 'count', # find the number of network type entries
                                     'date': 'first'})    # get the first date per group

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

The aggregation dictionary syntax is flexible and can be defined before the operation. You can also define functions inline using `lambda` functions to extract statistics that are not provided by the built-in options.

In [75]:
# Define the aggregation procedure outside of the groupby operation
aggregations = {
    'duration':'sum',
    'date': lambda x: max(x)
}
data.groupby('month').agg(aggregations)

,duration,date
month,,
2014-11,26639.441,2014-11-13 22:31:00
2014-12,14641.870,2014-12-14 19:54:00
2015-01,18223.299,2015-01-14 23:36:00
2015-02,15522.299,2015-02-12 06:58:00
2015-03,22750.441,2015-03-14 00:16:00


In [76]:
# Group the data frame by month and item and extract a number of stats from each group
data.groupby(['month', 'item']).agg({'duration': [min, max, sum],      # find the min, max, and sum of the duration column
                                     'network_type': "count",       # find the number of network type entries
                                     'date': [min, 'first', 'nunique']})    # get the min, first, and number of unique dates per group

duration                       network_type                date  \
                  min        max        sum        count                 min   
month   item                                                                   
2014-11 call    1.000   1940.000  25547.000          107 2014-10-15 06:58:00   
        data   34.429     34.429    998.441           29 2014-10-15 06:58:00   
        sms     1.000      1.000     94.000           94 2014-10-16 22:18:00   
2014-12 call    2.000   2120.000  13561.000           79 2014-11-14 17:24:00   
        data   34.429     34.429   1032.870           30 2014-11-13 06:58:00   
        sms     1.000      1.000     48.000           48 2014-11-14 17:28:00   
2015-01 call    2.000   1859.000  17070.000           88 2014-12-15 20:03:00   
        data   34.429     34.429   1067.299           31 2014-12-13 06:58:00   
        sms     1.000      1.000     86.000           86 2014-12-15 19:56:00   
2015-02 call    1.000   1863.000  14416.000           67 2015-01-15 10:36:00   
        data   34.429     34.429   1067.299           31 2015-01-13 06:58:00   
        sms     1.000      1.000     39.000           39 2015-01-15 12:23:00   
2015-03 call    2.000  10528.000  21727.000           47 2015-02-12 20:15:00   
        data   34.429     34.429    998.441           29 2015-02-13 06:58:00   
        sms     1.000      1.000     25.000           25 2015-02-19 18:46:00   

                                          
                           first nunique  
month   item                              
2014-11 call 2014-10-15 06:58:00     104  
        data 2014-10-15 06:58:00      29  
        sms  2014-10-16 22:18:00      79  
2014-12 call 2014-11-14 17:24:00      76  
        data 2014-11-13 06:58:00      30  
        sms  2014-11-14 17:28:00      41  
2015-01 call 2014-12-15 20:03:00      84  
        data 2014-12-13 06:58:00      31  
        sms  2014-12-15 19:56:00      58  
2015-02 call 2015-01-15 10:36:00      67  
        data 2015-01-13 06:58:00      31  
        sms  2015-01-15 12:23:00      27  
2015-03 call 2015-02-12 20:15:00      47  
        data 2015-02-13 06:58:00      29  
        sms  2015-02-19 18:46:00      17

In [93]:
grouped = data.groupby('month').agg({"duration": [min, max, 'mean']})
grouped.columns = grouped.columns.droplevel(level=0)
grouped.rename(columns={"min": "min_duration", "max": "max_duration", "mean": "mean_duration"}, inplace=True)
grouped.head()

,min_duration,max_duration,mean_duration
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


In [94]:
data.groupby('month').agg({"duration": [min, max, 'mean']})

duration                     
             min      max        mean
month                                
2014-11      1.0   1940.0  115.823657
2014-12      1.0   2120.0   93.260318
2015-01      1.0   1859.0   88.894141
2015-02      1.0   1863.0  113.301453
2015-03      1.0  10528.0  225.251891

However, this approach loses the original column names, leaving only the function names as column headers. A neater approach is using the `ravel()` method on the grouped columns. `Ravel()` turns a Pandas multi-index into a simpler array (of tuples, in this case), which we can combine into sensible column names:

In [101]:
grouped2 = data.groupby('month').agg({"duration": [min, max, 'mean']}) 
# Using ravel, and a string join, we can create better names for the columns:
grouped2.columns = ["_".join(x) for x in grouped2.columns.ravel()]
grouped2.head()
# grouped2.columns.ravel()

,duration_min,duration_max,duration_mean
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891
